##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Reentrenar un modelo de reconocimiento del habla con Model Maker de TensorFlow Lite


<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/speech_recognition"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/lite/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/lite/models/modify/model_maker/speech_recognition.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
</table>

En este bloc de colab, aprenderá a usar el [Model Maker de TensorFlow Lite](https://www.tensorflow.org/lite/models/modify/model_maker) para entrenar un modelo de reconocimiento del habla que pueda clasificar palabras habladas o frases cortas utilizando muestreos de sonido de un segundo. La librería Model Maker utiliza el aprendizaje por transferencia para volver a entrenar un modelo TensorFlow existente con un nuevo conjunto de datos, lo que reduce la cantidad de datos de muestra y el tiempo necesario para el entrenamiento.

De forma predeterminada, este bloc vuelve a entrenar el modelo (BrowserFft, del [reconocedor de comandos de voz TFJS](https://github.com/tensorflow/tfjs-models/tree/master/speech-commands#speech-command-recognizer)) usando un subconjunto de palabras del [conjunto de datos de comandos de voz ](https://www.tensorflow.org/datasets/catalog/speech_commands) (como "up", "down", "left" y "right"). A continuación, exporta un modelo TFLite que puede ejecutar en un dispositivo móvil o un sistema integrado (como una Raspberry Pi). También exporta el modelo entrenado como un TensorFlow SavedModel.

Este bloc también está diseñado para aceptar un conjunto de datos personalizado de archivos WAV, cargados en Colab en un archivo ZIP. Cuantas más muestras tenga para cada clase, mayor será su precisión, pero como el proceso de aprendizaje por transferencia usa incrustaciones de características del modelo preentrenado, aún puede obtener un modelo bastante preciso con sólo unas pocas docenas de muestreos en cada una de sus clases.

**Nota:** El modelo que vamos a entrenar está optimizado para el reconocimiento del habla con muestras de un segundo. Si desea realizar una clasificación de audio más genérica (como detectar diferentes tipos de música), le sugerimos que en su lugar siga [este Colab para volver a entrenar un clasificador de audio](https://colab.sandbox.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/audio_classification.ipynb).

Si desea ejecutar el bloc de notas con el conjunto de datos de habla predeterminado, puede ejecutarlo todo ahora haciendo clic en **Runtime &gt; Ejecutar todos** en la barra de herramientas de Colab. Sin embargo, si desea usar su propio conjunto de datos, continúe hasta [Preparar el conjunto de datos](#scrollTo=cBsSAeYLkc1Z) y siga las instrucciones que allí se indican.


### Importar los paquetes necesarios


Necesitará TensorFlow, Model Maker de TFLite y algunos módulos para manipulación de audio, reproducción y visualizaciones.

In [ ]:
!sudo apt -y install libportaudio2
!pip install tflite-model-maker

In [ ]:
import os
import glob
import random
import shutil

import librosa
import soundfile as sf
from IPython.display import Audio
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
from tflite_model_maker.config import ExportFormat

print(f"TensorFlow Version: {tf.__version__}")
print(f"Model Maker Version: {mm.__version__}")

## Preparar el conjunto de datos

Para realizar el entrenamiento con el conjunto de datos de habla predeterminado, basta con ejecutar todo el código siguiente tal cual.

Pero si desea entrenarse con su propio conjunto de datos de habla, siga estos pasos:

**Nota:** El modelo que volverá a entrenar espera que los datos de entrada sean aproximadamente un segundo de audio a 44.1 kHz. Model Maker realiza un remuestreo automático para el conjunto de datos de entrenamiento, por lo que no hay necesidad de remuestrear su conjunto de datos si tiene una frecuencia de muestreo distinta de 44.1 kHz. Pero tenga en cuenta que las muestras de audio de más de un segundo se dividirán en múltiples trozos de un segundo, y el trozo final se descartará si es inferior a un segundo.

1. Asegúrese de que cada muestra de su conjunto de datos está en formato de archivo **WAV, de aproximadamente un segundo de duración**. Después cree un archivo ZIP con todos sus archivos WAV, organizados en subcarpetas separadas para cada clasificación. Por ejemplo, cada muestra para un comando de voz "yes" debe estar en una subcarpeta llamada "yes". Incluso si sólo tiene una clase, las muestras deben guardarse en un subdirectorio con el nombre de la clase como nombre del directorio. Este script asume que su conjunto de datos **no está dividido** en conjuntos de entrenamiento/validación/prueba y realiza esa división por usted.
2. Haga clic en la pestaña **Archivos** del panel izquierdo y arrastre y suelte allí su archivo ZIP para cargarlo.
3. Utilice la siguiente opción desplegable para establecer **`use_custom_dataset`** en verdadero.
4. A continuación, pase a [Preparar un conjunto de datos de audio personalizado](#scrollTo=EobYerLQkiF1) para especificar el nombre de su archivo ZIP y el nombre del directorio del conjunto de datos.


In [ ]:
use_custom_dataset = False #@param ["False", "True"] {type:"raw"}

### Generar un conjunto de datos de ruido de fondo

Tanto si usa el conjunto de datos de habla predeterminado como uno personalizado, debe disponer de un buen conjunto de ruidos de fondo para que su modelo pueda distinguir el habla de otros ruidos (incluido el silencio).

Dado que las siguientes muestras de fondo se proporcionan en archivos WAV de un minuto de duración o más, tenemos que dividirlas en muestras más pequeñas de un segundo para poder reservar algunas para nuestro conjunto de datos de prueba. También combinaremos un par de fuentes de muestras diferentes para crear un conjunto completo de ruidos y silencios de fondo:

In [ ]:
tf.keras.utils.get_file('speech_commands_v0.01.tar.gz',
                        'http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz',
                        cache_dir='./',
                        cache_subdir='dataset-speech',
                        extract=True)
tf.keras.utils.get_file('background_audio.zip',
                        'https://storage.googleapis.com/download.tensorflow.org/models/tflite/sound_classification/background_audio.zip',
                        cache_dir='./',
                        cache_subdir='dataset-background',
                        extract=True)


**Nota:** Aunque hay una versión más reciente disponible, estamos usando la v0.01 del conjunto de datos de comandos de voz porque es una descarga más pequeña. La v0.01 incluye 30 comandos, mientras que la v0.02 añade cinco más ("backward", "forward", "follow", "learn" y "visual").

In [ ]:
# Create a list of all the background wav files
files = glob.glob(os.path.join('./dataset-speech/_background_noise_', '*.wav'))
files = files + glob.glob(os.path.join('./dataset-background', '*.wav'))

background_dir = './background'
os.makedirs(background_dir, exist_ok=True)

# Loop through all files and split each into several one-second wav files
for file in files:
  filename = os.path.basename(os.path.normpath(file))
  print('Splitting', filename)
  name = os.path.splitext(filename)[0]
  rate = librosa.get_samplerate(file)
  length = round(librosa.get_duration(filename=file))
  for i in range(length - 1):
    start = i * rate
    stop = (i * rate) + rate
    data, _ = sf.read(file, start=start, stop=stop)
    sf.write(os.path.join(background_dir, name + str(i) + '.wav'), data, rate)

### Preparar el conjunto de datos de comandos de voz

Ya hemos descargado el conjunto de datos de comandos de voz, así que ahora sólo tenemos que podar el número de clases para nuestro modelo.

Este conjunto de datos incluye más de 30 clasificaciones de órdenes del habla, y la mayoría de ellas tienen más de 2,000 muestras. Pero como estamos usando el aprendizaje por transferencia, no necesitamos tantas muestras. Así que el código siguiente hace algunas cosas:

- Especificar qué clasificaciones queremos usar y elimine el resto.
- Mantener sólo 150 muestras de cada clase para el entrenamiento (para demostrar que el aprendizaje por transferencia funciona bien con conjuntos de datos más pequeños y simplemente para reducir el tiempo de entrenamiento).
- Crear un directorio separado para un conjunto de datos de prueba para que podamos ejecutar fácilmente la inferencia con ellos más tarde.

In [ ]:
if not use_custom_dataset:
  commands = [ "up", "down", "left", "right", "go", "stop", "on", "off", "background"]
  dataset_dir = './dataset-speech'
  test_dir = './dataset-test'

  # Move the processed background samples
  shutil.move(background_dir, os.path.join(dataset_dir, 'background'))   

  # Delete all directories that are not in our commands list
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    name = os.path.basename(os.path.normpath(dir))
    if name not in commands:
      shutil.rmtree(dir)

  # Count is per class
  sample_count = 150
  test_data_ratio = 0.2
  test_count = round(sample_count * test_data_ratio)

  # Loop through child directories (each class of wav files)
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    files = glob.glob(os.path.join(dir, '*.wav'))
    random.seed(42)
    random.shuffle(files)
    # Move test samples:
    for file in files[sample_count:sample_count + test_count]:
      class_dir = os.path.basename(os.path.normpath(dir))
      os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
      os.rename(file, os.path.join(test_dir, class_dir, os.path.basename(file)))
    # Delete remaining samples
    for file in files[sample_count + test_count:]:
      os.remove(file)

### Preparar el conjunto de datos personalizado

Si desea entrenar el modelo con nuestro propio conjunto de datos de habla, deberá cargar sus muestras como archivos WAV en un ZIP ([como se ha descrito anteriormente](#scrollTo=cBsSAeYLkc1Z)) y modificar las siguientes variables para especificar su conjunto de datos:

In [ ]:
if use_custom_dataset:
  # Specify the ZIP file you uploaded:
  !unzip YOUR-FILENAME.zip
  # Specify the unzipped path to your custom dataset
  # (this path contains all the subfolders with classification names):
  dataset_dir = './YOUR-DIRNAME'

Tras cambiar el nombre del archivo y el nombre de la ruta anteriores, estará listo para entrenar el modelo con su conjunto de datos personalizado. En la barra de herramientas de Colab, seleccione **Runtime &gt; Ejecutar todo** para ejecutar todo el bloc de notas.

El código siguiente integra nuestras nuevas muestras de ruido de fondo en su conjunto de datos y, a continuación, separa una parte de todas las muestras para crear un conjunto de prueba.

In [ ]:
def move_background_dataset(dataset_dir):
  dest_dir = os.path.join(dataset_dir, 'background')
  if os.path.exists(dest_dir):
    files = glob.glob(os.path.join(background_dir, '*.wav'))
    for file in files:
      shutil.move(file, dest_dir)
  else:
    shutil.move(background_dir, dest_dir)

In [ ]:
if use_custom_dataset:
  # Move background samples into custom dataset
  move_background_dataset(dataset_dir)

  # Now we separate some of the files that we'll use for testing:
  test_dir = './dataset-test'
  test_data_ratio = 0.2
  dirs = glob.glob(os.path.join(dataset_dir, '*/'))
  for dir in dirs:
    files = glob.glob(os.path.join(dir, '*.wav'))
    test_count = round(len(files) * test_data_ratio)
    random.seed(42)
    random.shuffle(files)
    # Move test samples:
    for file in files[:test_count]:
      class_dir = os.path.basename(os.path.normpath(dir))
      os.makedirs(os.path.join(test_dir, class_dir), exist_ok=True)
      os.rename(file, os.path.join(test_dir, class_dir, os.path.basename(file)))
    print('Moved', test_count, 'images from', class_dir)

### Reproducir una muestra

Para asegurarnos de que el conjunto de datos parece correcto, juguemos con una muestra aleatoria del conjunto de prueba:

In [ ]:
def get_random_audio_file(samples_dir):
  files = os.path.abspath(os.path.join(samples_dir, '*/*.wav'))
  files_list = glob.glob(files)
  random_audio_path = random.choice(files_list)
  return random_audio_path

def show_sample(audio_path):
  audio_data, sample_rate = sf.read(audio_path)
  class_name = os.path.basename(os.path.dirname(audio_path))
  print(f'Class: {class_name}')
  print(f'File: {audio_path}')
  print(f'Sample rate: {sample_rate}')
  print(f'Sample length: {len(audio_data)}')

  plt.title(class_name)
  plt.plot(audio_data)
  display(Audio(audio_data, rate=sample_rate))

In [ ]:
random_audio = get_random_audio_file(test_dir)
show_sample(random_audio)

## Definir el modelo

Al usar Model Maker para reentrenar cualquier modelo, tiene que empezar por definir una especificación del modelo. La especificación define el modelo base a partir del cual su nuevo modelo extraerá las incrustaciones de características para empezar a aprender nuevas clases. La especificación de este reconocedor del habla se basa en el modelo [BrowserFft preentrenado de TFJS](https://github.com/tensorflow/tfjs-models/tree/master/speech-commands#speech-command-recognizer).

El modelo espera la entrada como una muestra de audio a 44.1 kHz y de algo menos de un segundo de duración: la longitud exacta de la muestra debe ser de 44034 fotogramas.

No necesita realizar ningún remuestreo con su conjunto de datos de entrenamiento. Model Maker se encarga de eso por usted. Pero cuando más tarde ejecute la inferencia, debe asegurarse de que su entrada coincide con ese formato esperado.

Todo lo que necesita hacer aquí es instanciar el [`BrowserFftSpec`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/BrowserFftSpec):


In [ ]:
spec = audio_classifier.BrowserFftSpec()

## Cargar su conjunto de datos 

Ahora necesita cargar su conjunto de datos de acuerdo con las especificaciones del modelo. Model Maker incluye la API [`DataLoader`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/DataLoader), que cargará su conjunto de datos desde una carpeta y se asegurará de que está en el formato esperado para las especificaciones del modelo.

Ya hemos reservado algunos archivos de prueba moviéndolos a un directorio separado, lo que facilita la ejecución posterior de la inferencia con ellos. Ahora crearemos un `DataLoader` para cada división: el conjunto de entrenamiento, el conjunto de validación y el conjunto de prueba.

#### Cargue el conjunto de datos de comandos de voz


In [ ]:
if not use_custom_dataset:
  train_data_ratio = 0.8
  train_data = audio_classifier.DataLoader.from_folder(
      spec, dataset_dir, cache=True)
  train_data, validation_data = train_data.split(train_data_ratio)
  test_data = audio_classifier.DataLoader.from_folder(
      spec, test_dir, cache=True)

#### Cargue un conjunto de datos personalizado

**Nota:** Configurar `cache=True` es importante para que el entrenamiento sea más rápido (sobre todo cuando hay que volver a muestrear el conjunto de datos), pero también requerirá más memoria RAM para mantener los datos. Si usa un conjunto de datos personalizado muy grande, el almacenamiento en caché podría superar su capacidad de RAM.

In [ ]:
if use_custom_dataset:
  train_data_ratio = 0.8
  train_data = audio_classifier.DataLoader.from_folder(
      spec, dataset_dir, cache=True)
  train_data, validation_data = train_data.split(train_data_ratio)
  test_data = audio_classifier.DataLoader.from_folder(
      spec, test_dir, cache=True)


## Entrenar el modelo


Ahora usaremos la función [`create()`](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/audio_classifier/create) de Model Maker para crear un modelo basado en nuestra especificación del modelo y el conjunto de datos de entrenamiento, y comenzar el entrenamiento.

Si está usando un conjunto de datos personalizado, es posible que desee cambiar el tamaño del lote según convenga al número de muestras de su conjunto de entrenamiento.

**Nota:** La primera época tarda más porque debe crear la caché. 

In [ ]:
# If your dataset has fewer than 100 samples per class,
# you might want to try a smaller batch size
batch_size = 25
epochs = 25
model = audio_classifier.create(train_data, spec, validation_data, batch_size, epochs)

## Revisar el rendimiento del modelo

Incluso si la precisión/pérdida parece buena a partir del resultado del entrenamiento anterior, es importante ejecutar también el modelo utilizando datos de prueba que el modelo no haya visto todavía, que es lo que hace aquí el método `evaluate()`:

In [ ]:
model.evaluate(test_data)

### Ver la matriz de confusión

Al entrenar un modelo de clasificación como éste, también es útil inspeccionar la [matriz de confusión](https://en.wikipedia.org/wiki/Confusion_matrix). La matriz de confusión le ofrece una representación visual detallada de lo bien que funciona su clasificador para cada clasificación en sus datos de prueba.

In [ ]:
def show_confusion_matrix(confusion, test_labels):
  """Compute confusion matrix and normalize."""
  confusion_normalized = confusion.astype("float") / confusion.sum(axis=1)
  sns.set(rc = {'figure.figsize':(6,6)})
  sns.heatmap(
      confusion_normalized, xticklabels=test_labels, yticklabels=test_labels,
      cmap='Blues', annot=True, fmt='.2f', square=True, cbar=False)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

confusion_matrix = model.confusion_matrix(test_data)
show_confusion_matrix(confusion_matrix.numpy(), test_data.index_to_label)

## Exportar el modelo

El último paso es exportar su modelo al formato TensorFlow Lite para su ejecución en dispositivos móviles/integrados y al formato [SavedModel](https://www.tensorflow.org/guide/saved_model) para su ejecución en cualquier otro lugar.

Al exportar un archivo `.tflite` desde Model Maker, éste incluye [metadatos del modelo](https://www.tensorflow.org/lite/inference_with_metadata/overview) que describen diversos detalles que pueden ayudar posteriormente durante la inferencia. Incluso incluye una copia del archivo de etiquetas de clasificación, por lo que no necesita un archivo `labels.txt` aparte (en la siguiente sección, mostramos cómo usar estos metadatos para ejecutar una inferencia).

In [ ]:
TFLITE_FILENAME = 'browserfft-speech.tflite'
SAVE_PATH = './models'

In [ ]:
print(f'Exporing the model to {SAVE_PATH}')
model.export(SAVE_PATH, tflite_filename=TFLITE_FILENAME)
model.export(SAVE_PATH, export_format=[mm.ExportFormat.SAVED_MODEL, mm.ExportFormat.LABEL])

## Ejecutar la inferencia con el modelo TF Lite

Ahora su modelo TFLite puede ser implementado y ejecutado usando cualquiera de las [bibliotecas de inferencia](https://www.tensorflow.org/lite/guide/inference) soportadas o con la nueva [API de tareas del AudioClassifier de TFLite](https://www.tensorflow.org/lite/inference_with_metadata/task_library/audio_classifier). El siguiente código muestra cómo puede ejecutar la inferencia con el modelo `.tflite` en Python.

In [ ]:
# This library provides the TFLite metadata API
! pip install -q tflite_support

In [ ]:
from tflite_support import metadata
import json

def get_labels(model):
  """Returns a list of labels, extracted from the model metadata."""
  displayer = metadata.MetadataDisplayer.with_model_file(model)
  labels_file = displayer.get_packed_associated_file_list()[0]
  labels = displayer.get_associated_file_buffer(labels_file).decode()
  return [line for line in labels.split('\n')]

def get_input_sample_rate(model):
  """Returns the model's expected sample rate, from the model metadata."""
  displayer = metadata.MetadataDisplayer.with_model_file(model)
  metadata_json = json.loads(displayer.get_metadata_json())
  input_tensor_metadata = metadata_json['subgraph_metadata'][0][
          'input_tensor_metadata'][0]
  input_content_props = input_tensor_metadata['content']['content_properties']
  return input_content_props['sample_rate']

Para observar lo bien que funciona el modelo con muestras reales, ejecute el siguiente bloque de código una y otra vez. Cada vez, obtendrá una nueva muestra de prueba y ejecutará la inferencia con ella, y podrá escuchar la muestra de audio a continuación.

In [ ]:
# Get a WAV file for inference and list of labels from the model
tflite_file = os.path.join(SAVE_PATH, TFLITE_FILENAME)
labels = get_labels(tflite_file)
random_audio = get_random_audio_file(test_dir)

# Ensure the audio sample fits the model input
interpreter = tf.lite.Interpreter(tflite_file)
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_size = input_details[0]['shape'][1]
sample_rate = get_input_sample_rate(tflite_file)
audio_data, _ = librosa.load(random_audio, sr=sample_rate)
if len(audio_data) < input_size:
  audio_data.resize(input_size)
audio_data = np.expand_dims(audio_data[:input_size], axis=0)

# Run inference
interpreter.allocate_tensors()
interpreter.set_tensor(input_details[0]['index'], audio_data)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

# Display prediction and ground truth
top_index = np.argmax(output_data[0])
label = labels[top_index]
score = output_data[0][top_index]
print('---prediction---')
print(f'Class: {label}\nScore: {score}')
print('----truth----')
show_sample(random_audio)

## Descargar el modelo TF Lite

Ahora puede implementar el modelo TF Lite en su dispositivo móvil o integrado. No necesita descargar el archivo de etiquetas porque, en su lugar, puede recuperar las etiquetas de los metadatos del archivo `.tflite`, como se muestra en el ejemplo de inferencia anterior.

In [ ]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(tflite_file)

Revise nuestras apps de ejemplo de extremo a extremo que realizan inferencias con modelos de audio TFLite en [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/android/) y [iOS](https://github.com/tensorflow/examples/tree/master/lite/examples/sound_classification/ios).